In [2]:
def dropPK():
    pk_drop = "ALTER TABLE result DROP CONSTRAINT result_pkey;"
    cursor.execute(pk_drop)
    conn.commit()

In [3]:
def addPK():
    pk_add = "ALTER TABLE result ADD CONSTRAINT result_pkey PRIMARY KEY(sid);"
    cursor.execute(pk_add)
    conn.commit()

In [4]:
def UPSERT():
    max_time = "select max(createtime) from result";
    cursor.execute(max_time);
    data = cursor.fetchone()
    if data[0] == None:
        bulkInsert = "insert into result (name,dept,sid,marks,total,percentage,status,date_of_exam,createtime,upttime,deleted) select * from student;"
        cursor.execute(bulkInsert)
        conn.commit()
    else:
        to_insert = f"select * from student where createtime>'{data[0]}';"
        to_update = f"select * from student where createtime<='{data[0]}';"
        cursor.execute(to_update)
        data = cursor.fetchall();
        #print(data)
        for i in range(len(data)):
            update_query = f"update result set name = '{data[i][0]}', upttime = '{data[i][8]}', deleted = '{data[i][10]}' where sid = {data[i][2]};"
            cursor.execute(update_query);
        conn.commit()
        cursor.execute(to_insert);
        data = cursor.fetchall();
        if(len(data)>0):
            dropPK();
            p = data[0][3]
            marks = "{"
            for k in p:
                marks+="{},".format(str(k))
            marks=marks[:-1]
            marks+="}"
            for i in range(len(data)):
                insert_query = "insert into result values('{}','{}',{},'{}',{},{},'{}','{}','{}','{}','{}');".format(
                    data[i][0],data[i][1],data[i][2],marks,data[i][4],data[i][5],data[i][6],data[i][7],data[i][8],data[i][9],
                    data[i][10])
                cursor.execute(insert_query)
            conn.commit()
            addPK()
    print("Success")